In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

pd.set_option('display.mpl_style', 'default')
plt.rcParams['figure.figsize'] = (150, 5)
plt.rcParams['font.family'] = 'sans-serif'
# Increase pandas width
pd.set_option('display.width', 190)
# Inline plots yay
%matplotlib inline

In [4]:
words_file = 'words.csv'
words_raw_data = pd.read_csv(words_file)

In [5]:
just_words = words_raw_data[words_raw_data.columns[5:]]
just_words = just_words.fillna(0)

## Extract Cooccurrences

In [69]:
def get_words(row):
    """Returns the list of the marted words in the given row"""
    return [ ind for ind, val in enumerate(row) if val > 0 ]

In [70]:
cooccs = {}
for row in just_words.values:
    words = get_words(row)
    for word in words:
        try: 
            cooccs[word]
        except KeyError:
            cooccs[word] = {}
        for coocc in words: 
            if coocc != word:     
                try:
                    cooccs[word][coocc] = cooccs[word][coocc] + 1
                except KeyError:
                    cooccs[word][coocc] = 1

In [76]:
def get_cooccurrences(column_index):
    return cooccs[column_index]

def get_cooccurrences_names(column_index):
    return [ just_words.columns[i] for i in cooccs[column_index] ]

In [77]:
get_cooccurrences_names(0)

['Aggressive',
 'Edgy',
 'Thoughtful',
 'Serious',
 'Good lyrics',
 'Unattractive',
 'Confident',
 'Youthful',
 'Boring',
 'Current',
 'Stylish',
 'Cheap',
 'Irrelevant',
 'Calm',
 'Outgoing',
 'Inspiring',
 'Beautiful',
 'Fun',
 'Authentic',
 'Credible',
 'Cool',
 'Catchy',
 'Sensitive',
 'Mainstream',
 'Superficial',
 'Annoying',
 'Passionate',
 'Not authentic',
 'Timeless',
 'Depressing',
 'Original',
 'Talented',
 'Distinctive',
 'Approachable',
 'Trendsetter',
 'Noisy',
 'Upbeat',
 'Relatable',
 'Energetic',
 'Sexy',
 'Over',
 'Fake',
 'Cheesy',
 'Superstar',
 'Intrusive',
 'Unoriginal',
 'Dated',
 'Unapproachable',
 'Classic',
 'Playful',
 'Arrogant',
 'Warm']